In [1]:
# Susie and Ryan

In [2]:
import pandas as pd
import numpy as np
import random
import altair as alt
import requests
import inspect
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import networkx as nx
import networkx.algorithms.community as nx_comm
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pyvis
from pyvis import network as net
from itertools import combinations
from community import community_louvain
from copy import deepcopy

In [2]:
# Main question: Is there a similarity in energy, danceability, and valence between the "Sociopath's Motivation" and "Offbrand Boyband" playlists?
# Secondary questions: Does danceability increase with energy (in individual playlists and with the 2 playlists combined
# Valence is a measure of happiness by Spotify, so we also wanted to analyze whether there is a relationship between energy and valence in the 2 playlists

In [3]:
# storing the credentials:
CLIENT_ID = "116bae2a86fd4737862816c5f45d4c36"
CLIENT_SECRET = "4f4a732d83d04cfa94acc26d2b77169f"
my_username = "sx47r9lq4dwrjx1r0ct9f9m09"

# instantiating the client
# source: Max Hilsdorf (https://towardsdatascience.com/how-to-create-large-music-datasets-using-spotipy-40e7242cc6a6)
client_credentials_manager = SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [4]:
#getting attributes
def get_audio_features_df(playlist):
    
    # Create an empty dataframe
    playlist_features_list = ["artist", "album", "track_name", "track_id","danceability","energy","key","loudness","mode", "speechiness","instrumentalness","liveness","valence","tempo", "duration_ms","time_signature"]
    playlist_df = pd.DataFrame(columns = playlist_features_list)
    
    # Loop through every track in the playlist, extract features and append the features to the playlist df
    for track in playlist["items"]:
        # Create empty dict
        playlist_features = {}
        # Get metadata
        playlist_features["artist"] = track["track"]["album"]["artists"][0]["name"]
        playlist_features["album"] = track["track"]["album"]["name"]
        playlist_features["track_name"] = track["track"]["name"]
        playlist_features["track_id"] = track["track"]["id"]
        
        # Get audio features
        audio_features = sp.audio_features(playlist_features["track_id"])[0]
        for feature in playlist_features_list[4:]:
            playlist_features[feature] = audio_features[feature]
        
        # Concat the DataFrames
        track_df = pd.DataFrame(playlist_features, index = [0])
        playlist_df = pd.concat([playlist_df, track_df], ignore_index = True)
        
    return playlist_df

In [5]:
#get playlist 1
boyband = pd.DataFrame(sp.user_playlist_tracks("susiepieper42", "5cdB8qvSa0V42uHE54W4X1"))
boyband

,href,items,limit,next,offset,previous,total
0,https://api.spotify.com/v1/playlists/5cdB8qvSa...,"{'added_at': '2022-09-26T16:05:53Z', 'added_by...",100,None,0,None,10
1,https://api.spotify.com/v1/playlists/5cdB8qvSa...,"{'added_at': '2022-09-26T16:06:30Z', 'added_by...",100,None,0,None,10
2,https://api.spotify.com/v1/playlists/5cdB8qvSa...,"{'added_at': '2022-09-26T16:06:47Z', 'added_by...",100,None,0,None,10
3,https://api.spotify.com/v1/playlists/5cdB8qvSa...,"{'added_at': '2022-09-26T16:08:41Z', 'added_by...",100,None,0,None,10
4,https://api.spotify.com/v1/playlists/5cdB8qvSa...,"{'added_at': '2022-09-26T16:08:54Z', 'added_by...",100,None,0,None,10
5,https://api.spotify.com/v1/playlists/5cdB8qvSa...,"{'added_at': '2022-09-26T16:09:19Z', 'added_by...",100,None,0,None,10
6,https://api.spotify.com/v1/playlists/5cdB8qvSa...,"{'added_at': '2022-09-26T16:10:33Z', 'added_by...",100,None,0,None,10
7,https://api.spotify.com/v1/playlists/5cdB8qvSa...,"{'added_at': '2022-09-26T16:12:10Z', 'added_by...",100,None,0,None,10
8,https://api.spotify.com/v1/playlists/5cdB8qvSa...,"{'added_at': '2022-09-26T16:13:29Z', 'added_by...",100,None,0,None,10
9,https://api.spotify.com/v1/playlists/5cdB8qvSa...,"{'added_at': '2022-09-26T17:28:26Z', 'added_by...",100,None,0,None,10


In [6]:
#getting features
boyband_features = get_audio_features_df(boyband)
boyband_features

,artist,album,track_name,track_id,danceability,energy,key,loudness,mode,speechiness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,"Crosby, Stills & Nash",Greatest Hits,Suite: Judy Blue Eyes,2Jf0PGy9NzR1PTXvRFfaoE,0.454,0.399,9,-11.558,1,0.0429,0,0.1820,0.322,154.244,444053,4
1,Simon & Garfunkel,Sounds Of Silence,April Come She Will,0djZ2ndRfAL69WYNra5jRC,0.489,0.186,9,-16.237,1,0.0290,0.000152,0.2000,0.349,102.245,109387,4
2,The Beach Boys,Smiley Smile (Remastered),Good Vibrations - Remastered 2001,5t9KYe0Fhd5cW6UYT4qP8f,0.396,0.473,10,-7.670,0,0.0345,0.000124,0.1090,0.385,133.449,219027,4
3,Florida Georgia Line,Here's To The Good Times,Cruise,5BhsEd82G0Mnim0IUH6xkT,0.457,0.948,10,-3.364,1,0.0354,0,0.0536,0.878,148.000,208960,4
4,The Chainsmokers,Paris,Paris,15vzANxN8G9wWfwAJLLMCg,0.646,0.644,2,-6.763,1,0.0310,0,0.0888,0.251,99.999,221520,4
5,MKTO,MKTO,American Dream,3JMAdPq5TUOKBGsTATjLEH,0.554,0.767,0,-5.043,1,0.0497,0,0.2310,0.497,122.846,225747,4
6,NEEDTOBREATHE,H A R D L O V E,HAPPINESS,7aag0fFgE9K1Gj8lJwQaGI,0.575,0.765,9,-6.608,1,0.0345,0,0.3310,0.407,98.037,205773,4
7,OneRepublic,Oh My My,Heaven,1lFHaHvZeLf6XrOqX6iLlS,0.585,0.724,10,-6.658,0,0.0321,0,0.1050,0.370,95.009,258947,4
8,Bastille,Quarter Past Midnight,Quarter Past Midnight,1etiUDkISHELzQGMY79ryt,0.504,0.821,8,-3.356,1,0.0598,0,0.0418,0.458,155.131,201799,4
9,"Earth, Wind & Fire","The Best Of Earth, Wind & Fire Vol. 1",September,2grjqo0Frpf2okIBiifQKs,0.697,0.832,9,-7.264,1,0.0298,0.00131,0.2690,0.979,125.926,215093,4


In [7]:
#get playlist 2
sociopath = pd.DataFrame(sp.user_playlist_tracks("ryankredell", "0dWk3ugBnoaztsCzY2WR4R"))
sociopath

,href,items,limit,next,offset,previous,total
0,https://api.spotify.com/v1/playlists/0dWk3ugBn...,"{'added_at': '2022-09-26T18:32:46Z', 'added_by...",100,None,0,None,9
1,https://api.spotify.com/v1/playlists/0dWk3ugBn...,"{'added_at': '2022-09-26T18:33:01Z', 'added_by...",100,None,0,None,9
2,https://api.spotify.com/v1/playlists/0dWk3ugBn...,"{'added_at': '2022-09-26T18:33:05Z', 'added_by...",100,None,0,None,9
3,https://api.spotify.com/v1/playlists/0dWk3ugBn...,"{'added_at': '2022-09-26T18:33:27Z', 'added_by...",100,None,0,None,9
4,https://api.spotify.com/v1/playlists/0dWk3ugBn...,"{'added_at': '2022-09-26T18:33:57Z', 'added_by...",100,None,0,None,9
5,https://api.spotify.com/v1/playlists/0dWk3ugBn...,"{'added_at': '2022-09-26T18:36:33Z', 'added_by...",100,None,0,None,9
6,https://api.spotify.com/v1/playlists/0dWk3ugBn...,"{'added_at': '2022-09-26T18:37:13Z', 'added_by...",100,None,0,None,9
7,https://api.spotify.com/v1/playlists/0dWk3ugBn...,"{'added_at': '2022-09-26T18:40:54Z', 'added_by...",100,None,0,None,9
8,https://api.spotify.com/v1/playlists/0dWk3ugBn...,"{'added_at': '2022-09-26T18:41:13Z', 'added_by...",100,None,0,None,9


In [8]:
#getting features
sociopath_features = get_audio_features_df(sociopath)
sociopath_features

,artist,album,track_name,track_id,danceability,energy,key,loudness,mode,speechiness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,DJ Khaled,"All I Do Is Win (feat. T-Pain, Ludacris, Snoop...","All I Do Is Win (feat. T-Pain, Ludacris, Snoop...",28jyTDJQyz5IeDTkB1lOuu,0.550,0.779,0,-3.808,1,0.210,0,0.3330,0.2930,150.068,230281,4
1,Drake,"Honestly, Nevermind",Jimmy Cooks (feat. 21 Savage),3F5CgOj3wFlRv51JsHbxhe,0.529,0.673,0,-4.711,1,0.175,0.000002,0.0930,0.3660,165.921,218365,4
2,Denzel Curry,Melt My Eyez See Your Future,Walkin,1q8DwZtQen5fvyB7cKbShC,0.557,0.654,11,-7.201,0,0.196,0,0.2910,0.3860,135.983,280000,4
3,Kanye West,My Beautiful Dark Twisted Fantasy,POWER,2gZUPNdnz5Y45eiGxpHGSc,0.542,0.914,0,-4.747,0,0.113,0,0.7440,0.5760,153.993,292093,4
4,Gloria Gaynor,20th Century Masters: The Millennium Collectio...,I Will Survive - Single Version,7cv28LXcjAC3GsXbUvXKbX,0.777,0.725,0,-8.681,1,0.050,0.00805,0.2460,0.5290,116.669,198066,4
5,Lil Wayne,Tha Carter III,A Milli,6ScJMrlpiLfZUGtWp4QIVt,0.674,0.695,6,-8.636,0,0.278,0.00202,0.1940,0.7730,151.486,221840,4
6,Eminem,The Eminem Show,Till I Collapse,4xkOaSrkexMciUUogZKVTS,0.548,0.847,1,-3.237,1,0.186,0,0.0816,0.1000,171.447,297787,4
7,Eminem,Curtain Call: The Hits (Deluxe Edition),"Lose Yourself - From ""8 Mile"" Soundtrack",5Z01UMMf7V1o0MzF86s6WJ,0.692,0.744,2,-4.565,1,0.257,0.000423,0.3590,0.0612,171.374,326467,4
8,BROCKHAMPTON,SATURATION III,BOOGIE,3NuN59jpH3B5cNa2M4r7XG,0.604,0.933,1,-3.034,1,0.214,0,0.3700,0.2800,110.029,193155,4


In [9]:
# getting playlists and features for larger sample, to be concatenated in next step
hot_miserable = pd.DataFrame(sp.user_playlist_tracks("313pk3sai2bekdmtoefrbjmbv5ey", "3wfcrfxMNAQZRnfjvSnoNv"))
night_out = pd.DataFrame(sp.user_playlist_tracks("316b7eir5onpduebous2uyrqwxue", "2Jts9QKHfyaGi0oeVFSeN0"))
chilly_friday = pd.DataFrame(sp.user_playlist_tracks("Sam + Zach", "1g44fvkwYsjasjPcy0HGOC"))
get_better = pd.DataFrame(sp.user_playlist_tracks("4uxd1wpjsxqvlpni1ixj09ipn", "5YS9zh4bCOpJqhenIJfFyw"))
popular_music = pd.DataFrame(sp.user_playlist_tracks("4uxd1wpjsxqvlpni1ixj09ipn", "4NC377bgf15Vwymy1yMR9A"))
obama_mixtape = pd.DataFrame(sp.user_playlist_tracks("jojoag", "4OsLXBh1nRf9638lyC0wuI"))
siri_selections = pd.DataFrame(sp.user_playlist_tracks("31zhcjb3h2e5miwmhyzeuj4cibmq", "6rao6zxPX8rBRZYQBtg0lh"))
kanye_sample = pd.DataFrame(sp.user_playlist_tracks("suyashdasgupta", "6r9PUfQYGPaBSnYve8cUHf"))



In [10]:
hot_miserable_features= get_audio_features_df(hot_miserable)
hot_miserable_features

night_out_features = get_audio_features_df(night_out)
night_out_features


chilly_friday_features = get_audio_features_df(chilly_friday)
chilly_friday_features

get_better_features = get_audio_features_df(get_better)
get_better_features

popular_music_features = get_audio_features_df(popular_music)
popular_music_features

obama_mixtape_features = get_audio_features_df(obama_mixtape)
obama_mixtape_features

siri_selections_features = get_audio_features_df(siri_selections)
siri_selections_features

kanye_sample_features = get_audio_features_df(kanye_sample)
kanye_sample_features

,artist,album,track_name,track_id,danceability,energy,key,loudness,mode,speechiness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,Various Artists,Rhythm and Soul - Sweet Love Songs,Bound,1C8rYFLZu5QFSN6KV18Q4E,0.458,0.540,5,-6.457,0,0.0270,0.000428,0.1150,0.6570,142.218,191205,4
1,Ms. Lauryn Hill,MTV Unplugged No. 2.0,Mystery of Iniquity - Live,5KZwRD3KklSP73jnLoNWtF,0.650,0.428,2,-10.998,0,0.6990,0.000000,0.7710,0.6260,74.300,310733,3
2,Mike Oldfield,Crises,In High Places,4RM7uN7fVEe89rt0f3NzcD,0.754,0.666,4,-9.516,0,0.0461,0.004540,0.0806,0.7500,156.571,213160,4
3,Smokey Robinson,Smokey,Will You Love Me Tomorrow?,3qdGPi6cpcOSAc1QzKFPmC,0.325,0.327,10,-12.301,1,0.0286,0.008990,0.1750,0.4130,145.669,280307,3
4,Bon Iver,Blood Bank,Woods,35zX9FPuA6qg9bqx79eskS,0.193,0.352,3,-10.821,0,0.0921,0.224000,0.1020,0.1620,77.129,285440,1
5,"Bobby ""Blue"" Bland",Dreamer,Ain't No Love In The Heart Of The City - Singl...,4qYHnP5AmKzXbJhciPV8si,0.518,0.376,9,-18.248,0,0.0431,0.860000,0.0976,0.7440,158.089,233560,4
6,Luther Vandross,Never Too Much,A House Is Not a Home,4gFVTfg1EFuuXJguJe1Z7W,0.374,0.236,5,-15.565,1,0.0317,0.000177,0.2470,0.0772,105.898,427693,4
7,Chaka Khan,I Feel for You,Through the Fire,7gh2v4IHnxdiwSgA6xluhe,0.563,0.593,8,-7.099,1,0.0322,0.000018,0.1450,0.4280,130.744,288653,4
8,Various Artists,Essential Blues Anthems,I Got a Woman,4n3jZ9O3ntnHrP1sjmWW8T,0.600,0.507,9,-5.729,1,0.1830,0.000022,0.1240,0.7310,198.488,171967,4
9,Daft Punk,Discovery,"Harder, Better, Faster, Stronger",5W3cjX2J3tjhG8zb6u0qHn,0.817,0.716,6,-8.898,0,0.1440,0.002630,0.3580,0.6920,123.474,224693,4


In [11]:
#concatenate and make 1 big list
large_list = pd.concat([sociopath_features,boyband_features,hot_miserable_features,night_out_features,chilly_friday_features,get_better_features,popular_music_features,obama_mixtape_features,siri_selections_features,kanye_sample_features])
large_list

,artist,album,track_name,track_id,danceability,energy,key,loudness,mode,speechiness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,DJ Khaled,"All I Do Is Win (feat. T-Pain, Ludacris, Snoop...","All I Do Is Win (feat. T-Pain, Ludacris, Snoop...",28jyTDJQyz5IeDTkB1lOuu,0.550,0.779,0,-3.808,1,0.2100,0,0.3330,0.2930,150.068,230281,4
1,Drake,"Honestly, Nevermind",Jimmy Cooks (feat. 21 Savage),3F5CgOj3wFlRv51JsHbxhe,0.529,0.673,0,-4.711,1,0.1750,0.000002,0.0930,0.3660,165.921,218365,4
2,Denzel Curry,Melt My Eyez See Your Future,Walkin,1q8DwZtQen5fvyB7cKbShC,0.557,0.654,11,-7.201,0,0.1960,0,0.2910,0.3860,135.983,280000,4
3,Kanye West,My Beautiful Dark Twisted Fantasy,POWER,2gZUPNdnz5Y45eiGxpHGSc,0.542,0.914,0,-4.747,0,0.1130,0,0.7440,0.5760,153.993,292093,4
4,Gloria Gaynor,20th Century Masters: The Millennium Collectio...,I Will Survive - Single Version,7cv28LXcjAC3GsXbUvXKbX,0.777,0.725,0,-8.681,1,0.0500,0.00805,0.2460,0.5290,116.669,198066,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,"Bobby ""Blue"" Bland",Dreamer,Ain't No Love In The Heart Of The City - Singl...,4qYHnP5AmKzXbJhciPV8si,0.518,0.376,9,-18.248,0,0.0431,0.86,0.0976,0.7440,158.089,233560,4
6,Luther Vandross,Never Too Much,A House Is Not a Home,4gFVTfg1EFuuXJguJe1Z7W,0.374,0.236,5,-15.565,1,0.0317,0.000177,0.2470,0.0772,105.898,427693,4
7,Chaka Khan,I Feel for You,Through the Fire,7gh2v4IHnxdiwSgA6xluhe,0.563,0.593,8,-7.099,1,0.0322,0.000018,0.1450,0.4280,130.744,288653,4
8,Various Artists,Essential Blues Anthems,I Got a Woman,4n3jZ9O3ntnHrP1sjmWW8T,0.600,0.507,9,-5.729,1,0.1830,0.000022,0.1240,0.7310,198.488,171967,4


In [12]:
# sorting by energy and danceability for each
# We wanted to get a high level view of energy and danceability in both playists, and to have it sorted 
# by the value. We hoped the visualization would guide our analysis

boyband_energy_sorted = boyband_features.sort_values(['energy'],ascending = [True])
boyband_danceability_sorted = boyband_features.sort_values(['danceability'],ascending = [True])
sociopath_energy_sorted = sociopath_features.sort_values(['energy'],ascending = [True])
sociopath_danceability_sorted =  sociopath_features.sort_values(['danceability'],ascending = [True])

In [23]:
boyband_energy_sorted

,artist,album,track_name,track_id,danceability,energy,key,loudness,mode,speechiness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
1,Simon & Garfunkel,Sounds Of Silence,April Come She Will,0djZ2ndRfAL69WYNra5jRC,0.489,0.186,9,-16.237,1,0.0290,0.000152,0.2000,0.349,102.245,109387,4
0,"Crosby, Stills & Nash",Greatest Hits,Suite: Judy Blue Eyes,2Jf0PGy9NzR1PTXvRFfaoE,0.454,0.399,9,-11.558,1,0.0429,0,0.1820,0.322,154.244,444053,4
2,The Beach Boys,Smiley Smile (Remastered),Good Vibrations - Remastered 2001,5t9KYe0Fhd5cW6UYT4qP8f,0.396,0.473,10,-7.670,0,0.0345,0.000124,0.1090,0.385,133.449,219027,4
4,The Chainsmokers,Paris,Paris,15vzANxN8G9wWfwAJLLMCg,0.646,0.644,2,-6.763,1,0.0310,0,0.0888,0.251,99.999,221520,4
7,OneRepublic,Oh My My,Heaven,1lFHaHvZeLf6XrOqX6iLlS,0.585,0.724,10,-6.658,0,0.0321,0,0.1050,0.370,95.009,258947,4
6,NEEDTOBREATHE,H A R D L O V E,HAPPINESS,7aag0fFgE9K1Gj8lJwQaGI,0.575,0.765,9,-6.608,1,0.0345,0,0.3310,0.407,98.037,205773,4
5,MKTO,MKTO,American Dream,3JMAdPq5TUOKBGsTATjLEH,0.554,0.767,0,-5.043,1,0.0497,0,0.2310,0.497,122.846,225747,4
8,Bastille,Quarter Past Midnight,Quarter Past Midnight,1etiUDkISHELzQGMY79ryt,0.504,0.821,8,-3.356,1,0.0598,0,0.0418,0.458,155.131,201799,4
9,"Earth, Wind & Fire","The Best Of Earth, Wind & Fire Vol. 1",September,2grjqo0Frpf2okIBiifQKs,0.697,0.832,9,-7.264,1,0.0298,0.00131,0.2690,0.979,125.926,215093,4
3,Florida Georgia Line,Here's To The Good Times,Cruise,5BhsEd82G0Mnim0IUH6xkT,0.457,0.948,10,-3.364,1,0.0354,0,0.0536,0.878,148.000,208960,4


In [24]:
alt.Chart(boyband_energy_sorted).mark_point().encode(
    y=alt.X("track_name", sort = None),
    x='energy'
)

alt.Chart(...)

In [25]:
alt.Chart(boyband_danceability_sorted).mark_point().encode(
    y=alt.X("track_name", sort = None),
    x='danceability'
)

alt.Chart(...)

In [26]:
alt.Chart(sociopath_energy_sorted).mark_point().encode(
    y=alt.X("track_name", sort = None),
    x='energy'
)

alt.Chart(...)

In [27]:
alt.Chart(sociopath_danceability_sorted).mark_point().encode(
    y=alt.X("track_name", sort = None),
    x='danceability'
)

alt.Chart(...)

In [28]:
# After trying this we realized that there wasn't a lot of information to be gained by doing this, so we 
# moved in a new direction

In [39]:
# Avg danceability, valence, and energy for each playlist
# We wanted to get a general idea of average levels of each of the variables across the playlists. 
# Both danceability and energy are higher for the "Sociopath's Motivation" playlist. The boyband playlist 
# has a higher average valence. 
boyband_avg_energy = boyband_features['energy'].mean()
boyband_avg_danceability = boyband_features['danceability'].mean()
sociopath_avg_energy = sociopath_features['energy'].mean()
sociopath_avg_danceability =sociopath_features['danceability'].mean()
boyband_avg_valence = boyband_features['valence'].mean()
sociopath_avg_valence =sociopath_features['valence'].mean()

## RF Fix for error in rounding.  Pass number and digits to function:
round(boyband_avg_energy, 2)

0.66

In [31]:
boyband_avg_danceability#.round(2)

0.5357000000000001

In [32]:
sociopath_avg_energy#.round(2)

0.7737777777777778

In [33]:
sociopath_avg_danceability#.round(2)

0.608111111111111

In [34]:
boyband_avg_valence#.round(2)

0.4896

In [35]:
sociopath_avg_valence#.round(2)

0.3738

In [40]:
# Danceability vs Energy for Boybands
# We wanted to analyze the relationships between our variables of interest in the playlists
# so we created scatter plots and calculated correlation coefficeints for each playlist and each pair of 
# variables

alt.Chart(boyband_features).mark_point().encode(
    y="energy",
    x='danceability'
)


alt.Chart(...)

In [41]:
# Correlation between danceability and energy for boybands

# We find for the boyband playlsit that there is a positive correlation between energy and danceability, 
# indicating that there is a slight increase in energy when danceability increases, for this particluar set of songs.
boyband_features['danceability'].corr(boyband_features['energy'])

0.3927190529523304

In [42]:
# Danceability vs Energy for Sociopaths
alt.Chart(sociopath_features).mark_point().encode(
    y="energy",
    x='danceability'
)



alt.Chart(...)

In [43]:
# Correlation between danceability and energy for sociopaths

# The correlation coefficient here is negative between energy and danceability, so for this particular playlist
# energy actually declines with danceabiliity. This is an interesting contract with the other playlist,
# and indicates that differing genres and other factors like mood may influence how energy and danceability
# are correlated. 
sociopath_features['danceability'].corr(sociopath_features['energy'])


-0.2310692954520078

In [45]:
#Energy vs. valence in boybands
alt.Chart(boyband_features).mark_point().encode(
    y="energy",
    x='valence'
)

alt.Chart(...)

In [47]:
boyband_features['energy'].corr(boyband_features['valence'])
#The above scatterplot and below correlation coefficient indicate a positive relationship between valence and energy
#When valence increases, energy also increases within the boyband playlist. 
#This follows our overall expectations within the wider genre of boyband music.

0.6059145084210796

In [48]:
#Energy vs. valence in sociopaths
alt.Chart(sociopath_features).mark_point().encode(
    y="energy",
    x='valence'
)

alt.Chart(...)

In [49]:
sociopath_features['valence'].corr(sociopath_features['energy'])
#The scatterplot and correlation show a negative relationship between energy and valence
#When valence increases, energy decreases within the sociopath playlist. 
#This is interesting when remembering the negative relationship between energy and danceability for the sociopath playlist.
#Again, this could probably be contributed to external factors like mood and overall genre.

-0.20065572537506138

In [50]:
#Danceability and valence in boybands
alt.Chart(boyband_features).mark_point().encode(
    y="danceability",
    x='valence')

alt.Chart(...)

In [51]:
boyband_features['danceability'].corr(boyband_features['valence'])
#In comparing danceability and valence in the boybands playlist, there is a slightly positive relationship between the two variables.
#When valence increases, danceability slightly increases. 
#This is fitting with our other results and the given associations with the genre.

0.24090518336657626

In [52]:
#Danceability and valence in sociopaths
alt.Chart(sociopath_features).mark_point().encode(
    y="danceability",
    x='valence')

alt.Chart(...)

In [53]:
sociopath_features['danceability'].corr(sociopath_features['valence'])
#Danceability and valence have a positive relationship in the sociopath playlist. 
#When valence increases, danceability also increases slightly. 
#This is a really interesting result, given the negative trends we've found throughout the sociopath playlist.


0.21872739478068795

In [54]:
# Finding correlation coefficients and creating graphs for the larger sample that we
# created earlier
alt.Chart(large_list).mark_point().encode(
    y="energy",
    x='danceability'
)


alt.Chart(...)

In [55]:
large_list['danceability'].corr(large_list['energy'])

0.4578053185753638

In [56]:
alt.Chart(large_list).mark_point().encode(
    y="energy",
    x='valence'
)

alt.Chart(...)

In [57]:
large_list['valence'].corr(large_list['energy'])

0.314494262394307

In [58]:
alt.Chart(large_list).mark_point().encode(
    y="danceability",
    x='valence'
)

alt.Chart(...)

In [42]:
large_list['danceability'].corr(large_list['valence'])

0.4999717108400522

In [43]:
# DANCEABILITY VS ENERGY: Large Sample
# The coefficient of the larger sample is .46 and is higher than the smaller lists
# (boyband (.39) and sociopath (-.2)). We expected boybands to be higher than sociopaths, but 
# it was interesting to see that they are still below the average correlation coefficient for a larger
# sample of playlists


# Energy vs Valence: Large Sample
# The coefficient of the larger sample is .31, compared to boyband (.6) and sociopath (-.2).
# Here "boyband" is much higher than the total sample, and both are much higher than sociopath. This mostly 
# shows that the sociopath playlist is much less happy than the other playlists. 


# Danceability vs Valence: Large Sample
# The coefficient of the larger sample is .50, compared to boyband (.24) and sociopath (.22).
# This shows that danceability are much more correlated in the larger sample, so in the larger sample
# songs that are danceable are more likely to be happy than in the sociopath and boyband lists. 
# We found this to be an interesting similarity because we wanted to find similarities between the boyband
# and sociopath playlists, and their close correlations are an intersting takeaway compared to the 
# larger sample. 


In [67]:
#Danceability, valence, and energy between our two playlists - a radar plot 
import plotly.graph_objects as go
import plotly.offline as pyo

feature_columns = ["danceability", "energy", "speechiness", "liveness", "instrumentalness", "liveness","valence", "danceability"]

def createRadarElement(row, feature_cols):
    return go.Scatterpolar(
        r = row[feature_cols].values.tolist(), 
        theta = feature_cols, 
        mode = 'lines', 
        name = row['track_name'])

def radar_for_lists(list_of_lists):
    for item in list_of_lists:
        this_list = pd.DataFrame(sp.playlist_items(item))
        audio_features_df = get_audio_features_df(this_list)
        length = len(audio_features_df)
        input_data = audio_features_df.sample(length).copy()
    
        data = list(input_data.apply(createRadarElement, axis=1, args=(feature_columns, )))  
        fig = go.Figure(data, )
        fig.write_image('fig.1.png')
        fig.show(renderer='iframe')
        # fig.show()

In [68]:
list_of_lists = ['0dWk3ugBnoaztsCzY2WR4R']

radar_for_lists(list_of_lists)

In [64]:
feature_columns = ["danceability", "energy", "speechiness", "liveness", "instrumentalness", "liveness","valence", "danceability"]

def createRadarElement(row, feature_cols):
    return go.Scatterpolar(
        r = row[feature_cols].values.tolist(), 
        theta = feature_cols, 
        mode = 'lines', 
        name = row['track_name'])

def radar_for_lists_2(list_of_lists):
    for item in list_of_lists:
        this_list = pd.DataFrame(sp.playlist_items(item))
        audio_features_df = get_audio_features_df(this_list)
        length = len(audio_features_df)
        input_data = audio_features_df.sample(length).copy()
    
        data = list(input_data.apply(createRadarElement, axis=1, args=(feature_columns, )))  
        fig = go.Figure(data, )
        fig.show(renderer='iframe')
#         fig.write_image('fig.2.png')

In [66]:
list_thing_2 = ['5cdB8qvSa0V42uHE54W4X1']
radar_for_lists_2(list_thing_2)

In [ ]:
# In the sociopath radar plot (figure 1) we see a connection between liveness and instrumentalness,
# which is driven by the track POWER. Otherwise, the shapes of the radars are similar, with the 
# sociopath radar being pulled more danceable in general. Valence is higher in a few tracks on
# the boyband playlist. These results fit with the initial graphs and correlations from earlier in our
# analysis. 